In [1]:
from FaceRec import Face_Recognition

File already exists:
	/Users/megankaye/Desktop/BeaverWorks/Work/Week2/DlibModels/dlib_models/dlib_face_recognition_resnet_model_v1.dat
File already exists:
	/Users/megankaye/Desktop/BeaverWorks/Work/Week2/DlibModels/dlib_models/shape_predictor_68_face_landmarks.dat


In [3]:
from database import Database

In [4]:
f = Face_Recognition()

In [5]:
d = Database("goodfile.npy")

In [6]:
f.name_faces_from_picture(d)

0.368553334808


['Megan Kaye']

In [ ]:
d.save_obj("goodfile.npy")

In [3]:

import collect_rss http://feeds.reuters.com/reuters/lifestyle lifestyle.pickle

SyntaxError: invalid syntax (<ipython-input-3-9b47f48b73bd>, line 2)

In [8]:
import pickle
z = pickle.load(open("C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/*.pickle", "rb"))


OSError: [Errno 22] Invalid argument: 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/*.pickle'

In [9]:
with open("C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/*.pickle", "rb") as f:
    p = pickle.load(f)
    

OSError: [Errno 22] Invalid argument: 'C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/*.pickle'

In [10]:
from bs4 import BeautifulSoup
import urllib.request

resp = urllib.request.urlopen("http://www.gpsbasecamp.com/national-parks")
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    print(link['href'])

/contact-gpsbasecamp.php
/privacy-policy.php
/terms-of-service.php
/
				National-Parks/map
/National-Historic-Parks
/National-Historic-Sites
/National-Monuments
/Other-NPS-Facilities
national-parks/Acadia_National_Park
national-parks/Arches_National_Park
national-parks/Badlands_National_Park
national-parks/Big_Bend_National_Park
national-parks/Biscayne_National_Park
national-parks/Black_Canyon_Of_The_Gunnison_National_Park
national-parks/Bryce_Canyon_National_Park
national-parks/Canyonlands_National_Park
national-parks/Capitol_Reef_National_Park
national-parks/Carlsbad_Caverns_National_Park
national-parks/Channel_Islands_National_Park
national-parks/Congaree_National_Park
national-parks/Crater_Lake_National_Park
national-parks/Cuyahoga_Valley_National_Park
national-parks/Death_Valley_National_Park
national-parks/Denali_National_Park_and_Preserve
national-parks/Dry_Tortugas_National_Park
national-parks/Everglades_National_Park
national-parks/Gates_Of_The_Arctic_National_Park_and_Preser

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [78]:
from bs4 import BeautifulSoup
import urllib.request
from search.collect_rss import collect

resp = urllib.request.urlopen("https://www.reuters.com/tools/rss")
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
print(soup)
for link in soup.find_all('a', href=True):
    l = link.find_all('a',  href=True)[0]["href"]
    p = collect(l, l + ".pickle")
    print(l)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


<!--[if !IE]> This has been served from cache <![endif]--><!--[if !IE]> Request served from apache server: prodie--i-0bdf42e7903dc9679 <![endif]--><!--[if !IE]> Cached on Mon, 31 Jul 2017 18:22:29 GMT and will expire on Mon, 31 Jul 2017 18:27:27 GMT <![endif]--><!--[if !IE]> token: 299b580b-1564-4604-949e-363e546ebd75 <![endif]--><!--[if !IE]> App Server /prodie--i-029d02a103883df8a/ <![endif]--><!DOCTYPE html>
<html lang="en"><head>
<title>Business &amp; Financial News, U.S &amp; International Breaking News  | Reuters</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="on" http-equiv="x-dns-prefetch-control"/><link href="//s1.reutersmedia.net" rel="dns-prefetch"/><link href="//s2.reutersmedia.net" rel="dns-prefetch"/><link href="//s3.reutersmedia.net" rel="dns-prefetch"/><link href="//s4.reutersmedia.net" rel="dns-prefetch"/><link href="//static.reuters.com" rel="dns-prefetch"/><link href="//www.googletagservices.com" rel="dns-prefetch"/

IndexError: list index out of range

In [65]:
#import urllib3
from bs4 import BeautifulSoup
import urllib.request
#import re

#connect to a URL

resp = urllib.request.urlopen("https://www.reuters.com/tools/rss")
soup = BeautifulSoup(response.data, 'html.parser')
#print(soup)
#print(soup.find_all("xmlLink"))
#soup.td["xmlLink"]
#print(soup)
#soup.find(class="xmlLink")

# #read html code
# html = website.read()

# #use re.findall to get all the links
#links = soup.findall(re.compile("((http|ftp)s?://.*?)"))
#z = soup.find_all('td')
print(soup.find_all('td', 'xmlLink'))

from bs4 import BeautifulSoup
import urllib.request

resp = urllib.request.urlopen("https://www.reuters.com/tools/rss")
soup = BeautifulSoup(response.data, 'html.parser')

for link in soup.find_all('td', 'xmlLink'):
    for link in soup.find_all('a', href=True):
    link.find_all('a',  href=True)[0]["href"]
#z = soup.td
#print(z)

[<td class="xmlLink"><a href="http://feeds.reuters.com/news/artsculture">Arts</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/businessNews">Business</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/companyNews">Company News</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/entertainment">Entertainment</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/environment">Environment</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/healthNews">Health News</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/lifestyle">Lifestyle</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/news/wealth">Money</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/MostRead">Most Read Articles</a></td>, <td class="xmlLink"><a href="http://feeds.reuters.com/reuters/oddlyEnoughNews">Oddly Enough</a></td>, <td class="xmlLink"><a href="http://feeds.reut

In [83]:
from bs4 import BeautifulSoup
import urllib.request
from search.collect_rss import collect

resp = urllib.request.urlopen("https://www.reuters.com/tools/rss")
soup = BeautifulSoup(response.data, 'html.parser')

for link in soup.find_all('td', 'xmlLink'):
    l = link.find_all('a',  href=True)[0]["href"]
    print(l)
    p = collect(l, l[l.rfind("/")+1:]+".pickle")

http://feeds.reuters.com/news/artsculture
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/uM1DTYoEbLo/us-czech-firepainter-idUSKBN1AD22C
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/RSyoQQYZDjc/us-bulgaria-graffiti-idUSKBN1AD1T5
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/bXDiC1lpkmk/us-germany-hitler-idUSKBN1AC390
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/g6Jy7gVMk7A/us-music-germany-wagner-idUSKBN1AA20D
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/mZZYpnRVcMM/us-exhibition-tulle-art-idUSKBN1AB2M3
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/dq45Er1EcSs/us-somalia-murals-idUSKBN1A920V
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/-9TcLiD9dlU/us-spain-dali-idUSKBN1A52LT
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/F2lOsynvHWU/us-exhibition-muppets-idUSKBN1A62OW
downloading: http://feeds.reuters.com/~r/news/artsculture/~3/aDfgxYPkiPs/us-britain-richardiii-idUSK

In [73]:
# link in soup.find_all('td', 'xmlLink'):
#     print(link)
link.find_all('a',  href=True)[0]["href"]
#link.

'http://feeds.reuters.com/reuters/USVideoWorldNews'

In [84]:
from search.entityDatabase import entityDatabase
edatb = entityDatabase()

edatb.add_Folder_Database("C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/pickles/*.pickle")


NameError: name 'glob' is not defined

In [91]:
import os

pickle_path = "C:/Users/User/Desktop/beaver/NumberWon/numberwon/alexa_skills/search/pickles"
            
for f in os.listdir(pickle_path): 
    if f.endswith(".pickle"):
        p = pickle.load(open(pickle_path+"/"+f, "rb"))

p

{'http://feeds.reuters.com/~r/Reuters/worldNews/~3/3KqFrg4-6Sk/us-britain-eu-may-idUSKBN1AG16X': 'LONDON (Reuters) - The automatic right of European Union citizens to live and work in Britain will end in March 2019 with Brexit, Prime Minister Theresa May\'s spokesman said on Monday, after her ministers publicly differed over the shape of the divorce with the EU.\n\nSince May\'s failed gamble on a snap election last month, the future of Brexit has been thrown into question with squabbling between her ministers over the pace, tone and terms of Britain\'s departure from the club it joined in 1973.\n\nMay, who on Monday interrupted a three-week holiday to attend a World War One commemoration ceremony, has faced public pressure to temper her plans for a clean break from the EU.\n\nThe level of discord is such that one opposition politician spoke of "civil war" within the government over Brexit and some of the bloc\'s most powerful politicians have even raised the prospect of Britain scrappi